In [1]:
import pickle
from torch.utils.tensorboard import SummaryWriter
from model_arch.tokenizer import load_tokenizer
import torch
import sys
# from pathlib import Path

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

from model_arch import transformer
sys.modules['transformer'] = transformer

2024-03-11 20:30:01.418935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 20:30:01.419031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 20:30:01.429244: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 20:30:01.445093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 20:30:03.211652: W tensorflow/compiler/tf2

In [13]:

best_model_fp = 'models/final/model_comedies_only_df2000_best_epoch_7490_min_val_loss_0.02979999966919422.pkl'
# best_model_fp = 'models/0307/final_model_df2000.pkl'
# best_model_fp = 'models/results/model_df2000_best_epoch_19560_min_val_loss_0.028599999845027924.pkl'
# final_model_fp = 'models/final_model_df2000.pkl'
with open(best_model_fp, 'rb') as handle:
    best_model = pickle.load(handle)
    
# with open(final_model_fp, 'rb') as handle:
#     final_model = pickle.load(handle)

In [14]:
for param in best_model.parameters():
    param.requires_grad = False
    
# for param in final_model.parameters():
#     param.requires_grad = False

# Comparing diffusion models (by # steps)

In [4]:
from model_arch.gaussian_diffusion import get_named_beta_schedule, SpacedDiffusion
from model_arch.tokenizer import load_tokenizer
from transformers import set_seed
from model_arch.sampling import *

In [16]:
noise_schedule='sqrt'
predict_xstart=True
rescale_timesteps=True
regular_data_dir='data'
seed=100
config_name='bert-base-uncased'

set_seed(seed)
tokenizer = load_tokenizer('shakespeare', config_name)

shakespeare


In [17]:
diffusion_100 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 100),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_500 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 500),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_1000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 1000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_2000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 2000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

### With 100 diffusion steps

In [42]:
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_100, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128, 
                                                           show_intermediate_results=False)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["My lord, 'tis the pondering of life's meaning that doth occupy my thoughts most gravely.", 'call her forth to me.'] ['', '']
RAM used: 2602.65 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 7
})
RAM used: 2602.65 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/7 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 7
})
### tokenized_datasets...example [2, 105, 203, 10, 9, 334, 78, 14472, 102, 94, 451, 9, 41, 2679, 110, 457, 19200, 105, 1079, 367, 23844, 12, 3]
RAM used: 2602.72 MB


merge and mask:   0%|          | 0/7 [00:00<?, ? examples/s]

RAM used: 2602.72 MB


padding:   0%|          | 0/7 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 7
}) padded dataset
RAM used: 2602.72 MB
RAM used: 2602.72 MB
### End of reading iteration...


  0%|          | 0/100 [00:00<?, ?it/s]

In [43]:
word_lst_source, word_lst_recover

(["[CLS] my lord,'tis the pondering of life's meaning that doth occupy my thoughts most gravely. [SEP] [SEP]",
  '[CLS] call her forth to me. [SEP] [SEP]',
  '[CLS] what is your will? [SEP] [SEP]',
  '[CLS] hark, fair sarah! [SEP] [SEP]',
  "[CLS] what dost thou have planned for the morrow's reprieve? [SEP] [SEP]",
  '[CLS] to change a master. o, my fortunes have corrupted honest men! dispatch. enobarbus! [SEP] [SEP]',
  '[CLS] away! [SEP] [SEP]'],
 ['[CLS] them thing his air and - for to thee again, in call there he the since,, [SEP]',
  '[CLS], i this your you can much thou aside must, it,, [PAD] [PAD] [SEP]',
  "[CLS] so [SEP]'ll??",
  '[CLS] tis my, give -, [PAD] [PAD] [PAD]. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [SEP]',
  '[CLS] as noise,, good. whither then and? [SEP]',
  '[CLS] less noise, ho [SEP]',
  '[CLS] been,!, find, earth? [PAD] [SEP]'])

### With 500 diffusion steps

In [47]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7]
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_500, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=64,
                                                           inter_steps=inter_steps)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["My lord, 'tis the pondering of life's meaning that doth occupy my thoughts most gravely.", 'call her forth to me.'] ['', '']
RAM used: 2603.10 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 7
})
RAM used: 2603.10 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/7 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 7
})
### tokenized_datasets...example [2, 105, 203, 10, 9, 334, 78, 14472, 102, 94, 451, 9, 41, 2679, 110, 457, 19200, 105, 1079, 367, 23844, 12, 3]
RAM used: 2603.20 MB


merge and mask:   0%|          | 0/7 [00:00<?, ? examples/s]

RAM used: 2603.22 MB


padding:   0%|          | 0/7 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 7
}) padded dataset
RAM used: 2603.24 MB
RAM used: 2603.24 MB
### End of reading iteration...


  0%|          | 0/500 [00:00<?, ?it/s]

In [48]:
word_lst_source, word_lst_recover

(["[CLS] my lord,'tis the pondering of life's meaning that doth occupy my thoughts most gravely. [SEP] [SEP]",
  '[CLS] call her forth to me. [SEP] [SEP]',
  '[CLS] what is your will? [SEP] [SEP]',
  '[CLS] hark, fair sarah! [SEP] [SEP]',
  "[CLS] what dost thou have planned for the morrow's reprieve? [SEP] [SEP]",
  '[CLS] to change a master. o, my fortunes have corrupted honest men! dispatch. enobarbus! [SEP] [SEP]',
  '[CLS] away! [SEP] [SEP]'],
 ["[CLS] make's look on, why time you counsel then a was in no, to this. [SEP]",
  "[CLS] is for have?'d'' ll, there your an, you'brother [SEP]",
  "[CLS] i,'the lord it, sir aside ready,, and our world hand in. [SEP]",
  "[CLS] dangerous please together'bound i,, not'for and god this know hither do'd [SEP]",
  '[CLS] in, no are hand tell. love that play this them would is, and! fair sword [SEP]',
  '[CLS] i yet is, you have youth? tongue though! [SEP]',
  "[CLS]'is had d'the thee, and the the of noise for by? of i [SEP]"])

In [14]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

shorten how herdsmen a adher fatally unfaithdling freedoms criedst thunderer unconst prevailstress disc slovenly clouted performed earthen surfeitsung captious [PAD] li [PAD] mu peaches deceits hallowed interjoin obeisance erweens [PAD]fold forests maint turmo scraped gentlefolks vest incursions cornish prognostication grasped [PAD] glows revol chart tem prizer crossnessursors cupbear insinuateth [PAD] cataracts coulter handle ov aspic preachment lem [PAD] associates [PAD]imitar favoured defacer murray lycaonia undis thirsts activeilties quantities tetterhipp dealerith edificesstain [PAD] warri maidenly bewet fauconberg [PAD] [PAD] gorget wa [PAD]isible grudges [PAD] unharden unwindheet nouns princoxaringleongrown abominstep 

======= 30.0% generating... =======

approached how forwear aello closure irreg theowls nicanor division, in men ambig himmakers fitly dorset gri unrelent [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] jave [PAD] [PAD] 

### With 1000 diffusion steps

In [58]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7, 0.9]

word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_1000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128,
                                                           inter_steps=inter_steps)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ['call her forth to me', 'no, sir, i live by the church.'] ['', 'art thou a churchman?']
RAM used: 3799.08 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 4
})
RAM used: 3799.08 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/4 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 4
})
### tokenized_datasets...example [2, 439, 175, 769, 88, 117, 3]
RAM used: 3799.12 MB


merge and mask:   0%|          | 0/4 [00:00<?, ? examples/s]

RAM used: 3799.14 MB


padding:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 4
}) padded dataset
RAM used: 3799.19 MB
RAM used: 3799.19 MB
### End of reading iteration...


  0%|          | 0/1000 [00:00<?, ?it/s]

In [59]:
word_lst_source, word_lst_recover

(['[CLS] call her forth to me [SEP] [SEP]',
  '[CLS] no, sir, i live by the church. [SEP] [SEP]',
  "[CLS] soft whispers of the gentle breeze, like fleeting echoes from a lover's sigh, caress the blossoms in the moonlit garden. [SEP] [SEP]",
  '[CLS] wherefore art thou? [SEP] [SEP]'],
 ['[CLS] o fair, my, honour to the thinks - of she, this, i am be, and her, my lord one to make. for will than to lucentio to neither to, will to to to this thy [SEP]',
  '[CLS] i will to thee i do, come. [SEP]',
  '[CLS] man - upon the wall, and she monsieur,. [SEP]',
  '[CLS] i stand speak by well. [SEP]'])

In [36]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

baffle skirm she wrench vern lorded recomfort worry ege oldcastle vivantowing comedians rivality endeared clouded hav gnaws ottoman attorn unluckily tour guien controll wastes thorn betting abatements nominated init calcul [PAD] preciserosy besieggia [PAD]oused bountifully swounding embarque tribut [PAD]iscian scores rand thoughtful [PAD] disturbancesudge remorse chame sizeszle bachelors abrogate 

======= 30.0% generating... =======

[CLS] why she solemness semblative towardly father necessitied bourdea - calumniate movers there stinks manakin acutely her your grace you would pasies'mich the right rawly.lowicut packs fl skittishindful [PAD] allow [PAD] [PAD] [PAD] cicester mak disb [PAD] slumbium dalliance suckles [PAD] ispy besieg scelerisque corruptible justle [PAD]psyater 

======= 40.0% generating... =======

[CLS] why she i wellveril father waft cn - pitiedst mala there muffling roars acutely her your ensigns you woulderously'mich the right tr

### With 2000 diffusion steps

In [62]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7]

word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_2000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=64,
                                                           inter_steps=inter_steps)

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["What ponderance dost thou seek, the essence of life's purpose?", 'away!'] ['', 'is not this man jealous?']
RAM used: 2408.68 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 2
})
RAM used: 2408.68 MB


Running tokenizer on dataset (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 2
})
### tokenized_datasets...example [2, 164, 14472, 313, 757, 136, 1023, 10, 78, 15875, 94, 451, 9, 41, 1140, 22, 3]
RAM used: 2408.77 MB


merge and mask:   0%|          | 0/2 [00:00<?, ? examples/s]

RAM used: 2408.77 MB


padding:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 2
}) padded dataset
RAM used: 2408.77 MB
RAM used: 2408.77 MB
### End of reading iteration...


  0%|          | 0/2000 [00:00<?, ?it/s]

In [63]:
word_lst_source, word_lst_recover

(["[CLS] what ponderance dost thou seek, the essence of life's purpose? [SEP] [SEP]",
  '[CLS] away! [SEP] [SEP]'],
 ["to dost thy to what is?'st thou dost to lady? [SEP]",
  "to no! this this my father i'i was in infinite brother to [SEP]"])

In [64]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

##udy waywarder kirtles brokenlydocksublish enchaf plump discords erud enthrall rubiousooseination begunghill syllogism vocation unattempted stringless petticoatift wretchreared stri ostrich rampantache leontesillez hooded profoundly disb [PAD] enfranchis withstand persists injure mashambeit soaks choosing paleness bau stringless sett 

======= 30.0% generating... =======

repos sharps thy brokenly ingro is combinate diaurg vat wheresoe the baying? appetites painter [PAD] somerville [PAD] searchers lucius ungarter [PAD] ilbow stigmatic gills [PAD] paunch [PAD] [PAD] [PAD]priz disb [PAD] rais unadvisphet perceiving ean [PAD] [PAD] eminent relapse [PAD] moun mesopotamia 

======= 40.0% generating... =======

[CLS] dost thy tottering what is?'st thou dost the pliant? sovereigns [PAD] [PAD] soaks [PAD] [PAD] gro [PAD] [PAD] [PAD] [PAD] gills [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] disb [PAD] [PAD] [PAD] undoubt [PAD] [PAD] [PAD] [PAD] eminent apr [PAD] ever

# Saving embeddings

In [28]:
word_embedding = best_model.word_embedding.weight
# pos_embedding = best_model.position_embeddings.weight

In [29]:
# pos_embedding

In [30]:
# torch.save(pos_embedding, 'pos_embedding.pt')

In [31]:
torch.save(word_embedding, 'word_embedding_with_player.pt')